In [39]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import constants
from scipy import stats
from scipy.optimize import leastsq

In [31]:
k = constants.value('Boltzmann constant in eV/K')
e = np.exp(1)

In [32]:
GRDF = pd.read_csv("../Results/sorted_data.csv")
GRDF = GRDF.drop(['Unnamed: 1'], axis = 1)

In [ ]:
GRDF["E"]     = np.NaN
GRDF["Eh"]    = np.NaN
GRDF["Eint"]  = np.NaN
GRDF["Ehint"] = np.NaN
GRDF["B0"]    = np.NaN
GRDF["Th"]    = np.NaN
GRDF["Tl"]    = np.NaN

for id in GRDF.NewID.unique():
    adjxVals = GRDF.adjTemp[GRDF.NewID == id].reset_index(drop = True)
    ldata    = GRDF.STVlogged[GRDF.NewID == id].reset_index(drop = True)

    split = np.argmax(ldata)  # split

    if split + 1 == len(ldata) or split == 0:
        lm1 = stats.linregress(x = adjxVals, y = ldata)
        lm2 = stats.linregress(x = adjxVals, y = ldata)
    else:
        try:
            lm1 = stats.linregress(x = adjxVals[:split], y = ldata[:split])
        except ValueError:
            lm1 = stats.linregress(x = adjxVals, y = ldata)
        try:
            lm2 = stats.linregress(x = adjxVals[split:], y = ldata[split:])
        except ValueError:
            lm2 = stats.linregress(x = adjxVals, y = ldata)

    GRDF.E[GRDF.NewID == id]  = lm1[0]  # slope of line right of Tpeak
    GRDF.Eh[GRDF.NewID == id] = lm2[0]  # slope of line left of Tpeak
    GRDF.Eint[GRDF.NewID == id]  = lm1[1]  # intercept of line right of Tpeak
    GRDF.Ehint[GRDF.NewID == id] = lm2[1]  # intercept of line left of Tpeak
    GRDF.B0[GRDF.NewID == id] = lm1[0]*(1/(k*283.15)) + lm1[1]  # ln(Bo) at 10C
    GRDF.Th[GRDF.NewID == id] = xVals[np.argmax(ldata)]  # Tpeak
    GRDF.Tl[GRDF.NewID == id] = min(xVals) # The lowest temp with STV
    # El I will bind so it must be lower than E in the NLLS

    GRDF.Eh[GRDF.Eh.isnull()] = GRDF.E[GRDF.Eh.isnull()]

In [54]:
def f(group):

    split = np.argmax(group.reset_index().STVlogged)  # split
    x = group.reset_index().adjTemp
    y = group.reset_index().STVlogged

    if split + 1 == len(y) or split == 0 or split ==1:
        lm1 = stats.linregress(x, y)
        lm2 = stats.linregress(x, y)
    else:
        try:
            lm1 = stats.linregress(x[:split], y[:split])
        except ValueError:
            lm1 = stats.linregress(x, y)
        try:
            lm2 = stats.linregress(x[split:], y[split:])
        except ValueError:
            lm2 = stats.linregress(x, y)
        
    vals = {"E"     : lm1[0],
            "Eh"    : lm2[0],
            "Eint"  : lm1[1],
            "Ehint" : lm2[1],
            "B0"    : lm1[0]*(1/(k*283.15)) + lm1[1],
            "Th"    : x[np.argmax(y)],
            "Tl"    : min(x)}
    
    return vals

In [55]:
GRDF.groupby("NewID").apply(f)

/home/david/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:106: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm


NewID
1       {'E': -1.10027619151, 'Eh': 1.22303160871, 'Ei...
3       {'E': -2.4272939661, 'Eh': 0.593559651569, 'Ei...
4       {'E': -2.1695159995, 'Eh': 0.737846687529, 'Ei...
5       {'E': -1.60113873173, 'Eh': 0.103964786473, 'E...
8       {'E': -1.04887434545, 'Eh': 1.585161711, 'Eint...
9       {'E': -1.37070530367, 'Eh': 0.789109246207, 'E...
10      {'E': -0.475537767788, 'Eh': 0.218364252449, '...
11      {'E': -0.544425232898, 'Eh': 0.548016655051, '...
12      {'E': -0.235768209826, 'Eh': 0.177605886995, '...
13      {'E': 0.044495903607, 'Eh': 0.044495903607, 'E...
14      {'E': -0.126385823174, 'Eh': 1.34008727211, 'E...
15      {'E': -1.06448180266, 'Eh': 0.952924586541, 'E...
16      {'E': -0.213678291256, 'Eh': 0.622375986078, '...
17      {'E': -0.0499298610339, 'Eh': 0.0559012142354,...
18      {'E': -0.0865800192039, 'Eh': 0.305346025727, ...
19      {'E': -0.436127709716, 'Eh': -0.436127709716, ...
20      {'E': -0.342723681519, 'Eh': 0.0796002379497, ...
21      

In [56]:
tmp = GRDF.groupby("NewID").apply(f)

/home/david/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:106: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm


In [72]:
tmp[1]

AttributeError: 'dict' object has no attribute 'B0'

In [51]:
split = np.argmax(tmp.reset_index().STVlogged)  # split
x = tmp.reset_index().adjTemp
y = tmp.reset_index().STVlogged

if split + 1 == len(y) or split == 0 or split == 1:
    lm1 = stats.linregress(x, y)
    lm2 = stats.linregress(x, y)
else:
    try:
        lm1 = stats.linregress(x[:split], y[:split])
    except ValueError:
        lm1 = stats.linregress(x, y)
    try:
        lm2 = stats.linregress(x[split:], y[split:])
    except ValueError:
        lm2 = stats.linregress(x, y)

vals = {"E"     : lm1[0],
        "Eh"    : lm2[0],
        "Eint"  : lm1[1],
        "Ehint" : lm2[1],
        "B0"    : lm1[0]*(1/(k*283.15)) + lm1[1],
        "Th"    : x[np.argmax(y)],
        "Tl"    : min(x)}

vals

{'B0': -10.819682871164794,
 'E': 0.75395322607412341,
 'Eh': 0.75395322607412341,
 'Ehint': -41.719442300000054,
 'Eint': -41.719442300000054,
 'Th': 41.800833574715476,
 'Tl': 40.536756389804452}

In [45]:
split

1

In [48]:
lm1 = stats.linregress(x, y)

In [49]:
lm1

LinregressResult(slope=0.75395322607412341, intercept=-41.719442300000054, rvalue=0.90678210824940941, pvalue=0.01262935106682286, stderr=0.1752716350596194)